In [1]:
# !pip install pymysql

In [2]:
import pymysql

In [3]:
con = pymysql.connect(
    host="localhost",
    port = 3306,
    user = 'root',
    password = '*tld5rk1vhfm',
    db='mldb2',
    charset='utf8'
)

In [4]:
con
print('1. 연결성공')

1. 연결성공


In [5]:
cur = con.cursor()
cur
print('2. 커서 객체 획득')

2. 커서 객체 획득


In [6]:
# sql = 'select * from member'
sql ='''
    SELECT m.name, m.tel, b.content
    from board b
    inner join member m on m.id=b.writer
    WHERE content like "%java%"
'''

In [7]:
result_count = cur.execute(sql)
result_count
print('3.sql 전송 성공', result_count)

3.sql 전송 성공 2


In [8]:
rows=cur.fetchall()
print('4.검색결과-----------')
print(rows)

4.검색결과-----------
(('apple', '555555555', 'fun java'), ('apple', '555555555', 'good java'))


In [9]:
con.close()

In [10]:
for index, row in enumerate(rows) :
    print(index, row[0])

0 apple
1 apple


### 함수로 만들기 

In [11]:
def connect() :
    con = pymysql.connect(
    host="localhost",
    port = 3306,
    user = 'root',
    password = '*tld5rk1vhfm',
    db='mldb2',
    charset='utf8'
)
    print('1. 연결성공')
    
    cur = con.cursor()
    print('2. 커서 객체 획득')
    
    return cur, con
    

In [12]:
cur2, con = connect()

1. 연결성공
2. 커서 객체 획득


In [13]:
cur2

In [14]:
con

In [15]:
def sql_execute(cur2, data) :
    sql = """
    select o2.orderid, m.name, o2.productid, o2.totalprice
    from orderlist o2
    inner join `member` m
    on o2.memberid = m.id and m.id = %s
    order by productid desc
    """
    
    result = cur2.execute(sql,data)
    return result 

In [16]:
sql_execute(cur2, 'park')

4

In [17]:
def out_data(cur2) :
    rows = cur2.fetchall()
    return rows

In [18]:
result_rows = out_data(cur2)
result_rows

((2, 'park', '101', 6000),
 (1, 'park', '100', 5000),
 (3, 'park', '100', 10000),
 (4, 'park', '100', 3000))

In [90]:
def db_close(con) :
    con.close()

In [91]:
db_close(con)

In [19]:
# 제일 높은 금액 ?

[x[3] for x in result_rows]

[6000, 5000, 10000, 3000]

In [20]:
max([x[3] for x in result_rows])

10000

In [21]:
# 제일 많이 산 물건의 id? 

import pandas as pd

In [22]:
result_df = pd.DataFrame(data=result_rows,
                        columns=['주문id','name','제품id','주문총가격'])

In [23]:
result_df

,주문id,name,제품id,주문총가격
0,2,park,101,6000
1,1,park,100,5000
2,3,park,100,10000
3,4,park,100,3000


In [24]:
[x for x in result_df['제품id']]

['101', '100', '100', '100']

In [25]:
from collections import Counter

In [26]:
counter_id = Counter(result_df['제품id'])

In [27]:
counter_id

Counter({'101': 1, '100': 3})

In [28]:
max_id = counter_id.most_common(n=1)
print(max_id)
print(max_id[0][0])
max_id_result = max_id[0][0]
max_id_result

[('100', 3)]
100


'100'

In [29]:
## 추천시스템 만들기 

def sql_execute2(cur2,data) :
    sql = """
    select * from product
    where id = %s
    """
    result2 = cur2.execute(sql, data)
    return result2

In [30]:
cur3, con = connect()
cur3

1. 연결성공
2. 커서 객체 획득


In [31]:
sql_result2 = sql_execute2(cur3,max_id_result)
sql_result2

1

In [32]:
result2_rows = out_data(cur3)
result2_rows[0]

('100', 'java', 'fun java', '1000', 1)

In [33]:
print('당신에게 추천하고 싶은 물건은 다음과 같습니다.')
print('----------------------------------------------')
print('물건의 이름은 ',result2_rows[0][1])
print('물건의 내용은 ',result2_rows[0][2])
print('물건의 가격은 ',result2_rows[0][3])

당신에게 추천하고 싶은 물건은 다음과 같습니다.
----------------------------------------------
물건의 이름은  java
물건의 내용은  fun java
물건의 가격은  1000


In [34]:
## 물건 구매하기 
def sql_execute3(con, cur2, data) :
    sql = """
    insert into orderlist values (null, %s,%s,%s)
    """
    result = cur2.execute(sql,data)
    con.commit()
    return result

In [35]:
order_question = input('구매하시겠습니까?(yes/no)')
if order_question == 'yes' :
    print('--------구매를 진행합니다.---------')
    your_id = input('당신의 id는')
    data_list =[your_id, result2_rows[0][0], int(result2_rows[0][3])]
    result3 = sql_execute3(con,cur2,data_list)
    
    if result3 == 1 :
        print('구매완료')
    else : 
        print('구매실패')

구매하시겠습니까?(yes/no)yes
--------구매를 진행합니다.---------
당신의 id는park
구매완료


In [36]:
con.close()

### minimini project

추천 알고리즘 : 사용자가 가장 최근 구매한 물품의 카테고리와 같은 제품을 추천한다.

In [37]:
def connect():
    con = pymysql.connect(host='localhost' , 
                          port=3306 , user ='root', 
                          password = '*tld5rk1vhfm', db='mldb2',charset = 'utf8')
    print('1.연결성공')
    cur=con.cursor()
    print('2. 커서객체 획득')
    return cur, con

In [38]:
cur4, con = connect()

1.연결성공
2. 커서객체 획득


In [39]:
def sql_execute4(cur, data):
    sql = """
    select o2.orderid, m.name, o2.productid, p.category
    from orderlist o2
    inner join `member` m 
    on o2.memberid = m.id and m.id = %s 
    inner join product p 
    on o2.productid = p.id 
    order by productid desc
    """
    result = cur.execute(sql,data)
    return result 

In [40]:
result4 = sql_execute4(cur4, 'park')

In [41]:
result_rows4 = out_data(cur4)

In [42]:
result_rows4

((2, 'park', '101', 2),
 (1, 'park', '100', 1),
 (3, 'park', '100', 1),
 (4, 'park', '100', 1))

In [43]:
df = pd.DataFrame(result_rows4 , columns=['orderid','이름','물건id','물건 카테고리'])
df

,orderid,이름,물건id,물건 카테고리
0,2,park,101,2
1,1,park,100,1
2,3,park,100,1
3,4,park,100,1


In [44]:
latest_id = df[df.orderid == max(df.orderid)]['물건 카테고리'].values
latest_id[0]

1

In [45]:
def sql_excute5(cur,data):
    sql="""
    select * from product p
    where p.category = %s
    """
    result= cur.execute(sql,data)
    return result

In [46]:
sql_excute5(cur4,latest_id[0])

3

In [47]:
recommend = out_data(cur4)
recommend

(('100', 'java', 'fun java', '1000', 1),
 ('104', 'swift', 'fun swift', '5000', 1),
 ('108', 'elk', 'fun elk', '9000', 1))

In [48]:
len(recommend)

3

In [49]:
print('당신에게 추천하고 싶은 물건은 다음과 같습니다.')
print('-----------------------------------------------')

for i in recommend :
    print('추천 제품 :', i[1], '\t 가격 : ',i[3])


당신에게 추천하고 싶은 물건은 다음과 같습니다.
-----------------------------------------------
추천 제품 : java 	 가격 :  1000
추천 제품 : swift 	 가격 :  5000
추천 제품 : elk 	 가격 :  9000


In [50]:
def sql_excute6(con ,cur , data):
    sql = """
    insert into orderlist values (null,%s,%s,%s)
    """
    result = cur.execute(sql,data)
    con.commit()
    return result

In [53]:
n=0
while n==0:
    order_question = input('몇번제품 구매하시겠습니까? (싫으면 n) >>> ')
    if order_question !='n':
        print('-----구매를 진행합니다. ----------')
        your_id= input('당신의 id는 ')
        
        data_list=[your_id, recommend[int(order_question)-1][0], int(recommend[int(order_question)-1][3])]
        result5= sql_excute6(con,cur4,data_list)
        if result5==1:
            print('-------구매 완료---------')
        else:
            print('------구매 문제 발생')
    else:
        n=1

몇번제품 구매하시겠습니까? (싫으면 n) >>> 2
-----구매를 진행합니다. ----------
당신의 id는 park
-------구매 완료---------
몇번제품 구매하시겠습니까? (싫으면 n) >>> 1
-----구매를 진행합니다. ----------
당신의 id는 park
-------구매 완료---------
몇번제품 구매하시겠습니까? (싫으면 n) >>> n
